In [1]:
# importig required packages
import matplotlib as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns


In [2]:
salary_df = pd.read_csv("salary .csv")
salary_df.shape

(514, 11)

In [3]:
salary_df.describe()

,salary,exprior,yearsworked,yearsrank,market,degree,otherqual,position,male,Field,yearsabs
count,513.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000
mean,50863.220098,2.926070,12.852140,7.052529,0.948521,0.964981,0.044747,2.132296,0.750973,2.529183,6.982490
std,12685.132358,4.791397,9.444695,6.414771,0.149380,0.184008,0.206950,0.820075,0.432871,1.127420,16.873156
min,29000.000000,0.000000,0.000000,0.000000,0.710000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,40000.000000,0.000000,4.000000,2.000000,0.840000,1.000000,0.000000,1.000000,1.000000,2.000000,0.000000
50%,50096.000000,0.500000,12.000000,5.000000,0.920000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000
75%,60345.000000,4.000000,22.000000,12.000000,1.020000,1.000000,0.000000,3.000000,1.000000,4.000000,2.000000
max,96156.000000,25.000000,41.000000,28.000000,1.330000,1.000000,1.000000,3.000000,1.000000,4.000000,118.000000


In [9]:
#checking for duplicates
duplicates = salary_df['salary'].is_unique
duplicates

False

# removal of duplicates
def duplicate_removal():
    first = len(salary_df.drop_duplicates(subset='salary', keep='first'))
    return first - duplicates
duplicate_removal

In [4]:
# checking for null values
salary_df.isnull().sum()

salary         1
exprior        0
yearsworked    0
yearsrank      0
market         0
degree         0
otherqual      0
position       0
male           0
Field          0
yearsabs       0
dtype: int64

In [ ]:
def duplicate

### Splitting the dataset into Training set and Test set


In [ ]:

#dependant variable is salary as the change in years should affect the salary earned 
yrs_worked = salary_df.iloc[:, 2].values
# independant variable is years worked  
salary_earned = salary_df.iloc[:, 0].values
yrs_worked

In [6]:
#histogram
sns.set()
plt.figure(figsize=(15,10))
plt.hist(salary_df['salary'], color='r', stacked=False, orientation='vertical')
plt.ylabel('salary')
plt.xlabel('number of employees')
plt.title('Salaries Of Employees')
plt.show()

TypeError: 'module' object is not callable